<a href="https://colab.research.google.com/github/Architgupta20/eccentric-assignment/blob/main/ArchitGupta_70282300005.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing torch + torchvision (CUDA 12.1 build)
!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121

In [ ]:
# Installing diffusion + helper libraries:
!pip install -q diffusers transformers accelerate safetensors controlnet_aux xformers

In [ ]:
# Login to Hugging Face
from huggingface_hub import login
from getpass import getpass

# Please paste your Hugging Face Token:
login(token=getpass("Paste your Hugging Face token: "))

Paste your Hugging Face token: ··········


In [ ]:
import os, json, random, torch
from diffusers import StableDiffusionXLPipeline, StableDiffusionXLImg2ImgPipeline, UniPCMultistepScheduler

In [ ]:
# Define model
model_id = "stabilityai/stable-diffusion-xl-base-1.0"

# Load base pipeline
pipe = StableDiffusionXLPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True
).to("cuda")

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
print("✅ SDXL base model loaded")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

text_encoder/model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

text_encoder_2/model.safetensors:   0%|          | 0.00/2.78G [00:00<?, ?B/s]

...

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


✅ SDXL base model loaded


In [ ]:
locations = {
    "Bandra_Worli_Sea_Link": [
    "Bandra-Worli Sea Link, Mumbai, dawn aerial view, mist hovering above the water, golden sunlight glinting on bridge cables, a few cars leaving glowing streaks, city skyline fading into soft clouds, captured with a 24mm wide-angle DSLR lens, cinematic HD photograph",
    "Bandra-Worli Sea Link, Mumbai, twilight side view, reflections of neon city lights dancing on calm waves, rain-slick bridge surface, motorcycles and cars leaving motion trails, subtle fog around distant towers, shot with 50mm lens, imaginative cinematic HD photo",
    "Bandra-Worli Sea Link, Mumbai, close-up of bridge suspension cables in the soft orange glow of sunset, tiny birds perched on cables, reflections of the sky in rippling water below, shallow depth-of-field using 85mm telephoto lens, magical realistic HD photograph"
    ],

    "Hawa_Mahal_Jaipur": [
    "Hawa Mahal, Jaipur, front view at sunrise, soft pink sandstone glowing warmly, morning mist weaving between balconies, a street musician playing in the background, pigeons flying across the facade, captured with 35mm DSLR lens, dreamy HD photo",
    "Hawa Mahal, Jaipur, side-top angle, sunbeams catching carved windows, bustling market stalls casting colorful shadows, local people walking with umbrellas, rooftops glowing in golden light, 24mm wide-angle lens, cinematic HD photograph",
    "Hawa Mahal, Jaipur, close-up of upper jharokhas in late morning, sunlight streaming through intricate carvings creating dramatic shadows, small group of tourists pausing to admire details, shallow depth-of-field with 85mm lens, artistic HD photo"
    ],

    "Nagpur_Rainforest": [
    "Nagpur rainforest, aerial view at dawn, mist curling through the dense treetops like soft clouds, sunlight breaking in golden beams, occasional birds gliding through the canopy, captured with drone and 16mm wide-angle lens, magical HD landscape photo",
    "Nagpur rainforest, ground-level trail with small stream, sunlight filtering through thick leaves forming natural spotlights, wet rocks sparkling, a deer glimpsed in the shadows, shot with 50mm lens, immersive HD close-up",
    "Nagpur rainforest, wide-angle view during golden afternoon, fog lifting from distant trees revealing layered greens, tiny waterfall in the distance, light rays piercing canopy gaps, captured with 24mm wide-angle DSLR lens, cinematic HD landscape photo"
    ]
}

print("✅ Prompts ready: highly imaginative, cinematic, HD, realistic, location-specific, and unique.")

✅ Prompts ready: highly imaginative, cinematic, HD, realistic, location-specific, and unique.


In [ ]:
# Generating 9 images (3 per location) and saving the metadata

import os, json, random, torch

# Create output folder
output_dir = "eccentric_outputs"
os.makedirs(output_dir, exist_ok=True)

metadata = []  # to store reproducibility details

for loc, prompts in locations.items():
    # Create sub-folder for each location
    loc_folder = os.path.join(output_dir, loc)
    os.makedirs(loc_folder, exist_ok=True)

    for i, prompt in enumerate(prompts):
        # Use a fixed random seed for reproducibility
        seed = random.randint(1, 2**31-1)
        generator = torch.Generator("cuda").manual_seed(seed)

        # Generate the image
        image = pipe(
            prompt=prompt,
            negative_prompt="blurry, low quality, distorted, unrealistic",
            num_inference_steps=40,   # steps to refine image
            guidance_scale=7.5,       # how strongly the model follows prompt
            generator=generator
        ).images[0]

        # Save image
        filename = f"{loc}_view_{i+1}.png"
        filepath = os.path.join(loc_folder, filename)
        image.save(filepath)

        # Save metadata
        metadata.append({
            "location": loc,
            "view": i+1,
            "prompt": prompt,
            "seed": seed,
            "file": filepath,
            "model": model_id,
            "steps": 70,
            "guidance": 7.5
        })

# Save metadata JSON file
with open(os.path.join(output_dir, "metadata.json"), "w") as f:
    json.dump(metadata, f, indent=2)

print("✅ Generated 9 images and saved metadata in eccentric_outputs/")

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

✅ Generated 9 images and saved metadata in eccentric_outputs/


In [ ]:
!rm -rf eccentric_outputs

In [15]:
!zip -r eccentric_outputs.zip eccentric_outputs

  adding: eccentric_outputs/ (stored 0%)
  adding: eccentric_outputs/Nagpur_Rainforest/ (stored 0%)
  adding: eccentric_outputs/Nagpur_Rainforest/Nagpur_Rainforest_view_1.png (deflated 0%)
  adding: eccentric_outputs/Nagpur_Rainforest/Nagpur_Rainforest_view_2.png (deflated 0%)
  adding: eccentric_outputs/Nagpur_Rainforest/Nagpur_Rainforest_view_3.png (deflated 0%)
  adding: eccentric_outputs/Hawa_Mahal_Jaipur/ (stored 0%)
  adding: eccentric_outputs/Hawa_Mahal_Jaipur/Hawa_Mahal_Jaipur_view_2.png (deflated 0%)
  adding: eccentric_outputs/Hawa_Mahal_Jaipur/Hawa_Mahal_Jaipur_view_1.png (deflated 0%)
  adding: eccentric_outputs/Hawa_Mahal_Jaipur/Hawa_Mahal_Jaipur_view_3.png (deflated 0%)
  adding: eccentric_outputs/Bandra_Worli_Sea_Link/ (stored 0%)
  adding: eccentric_outputs/Bandra_Worli_Sea_Link/Bandra_Worli_Sea_Link_view_2.png (deflated 0%)
  adding: eccentric_outputs/Bandra_Worli_Sea_Link/Bandra_Worli_Sea_Link_view_1.png (deflated 0%)
  adding: eccentric_outputs/Bandra_Worli_Sea_Link/